In [ ]:
import pandas as pd
import geopandas as gpd

# 1. โหลดข้อมูลแปลงเกษตร
farms = gpd.read_file(r"E:\PythonFor_Flood\drive-download-20260119T075939Z-3-001\พระนครศรีอยุธยา.shp")

# 1.1 กรองข้อมูลก่อนไปขั้นตอนอื่น
farms = farms[farms['a_name'] == 'ผักไห่'].copy()

# กรองเฉพาะอำเภอที่ต้องการ เช่น บางปะอิน และ ผักไห่ ['บางปะอิน', 'ผักไห่']
# target_districts = ['ผักไห่']
# farms = farms[farms['a_name'].isin(target_districts)].copy()

print(f"จำนวนแปลงเกษตรในกลุ่มอำเภอที่เลือก: {len(farms)} แปลง")
print(f"จำนวนแปลงเกษตรตั้งต้น: {len(farms)} แปลง")

จำนวนแปลงเกษตรในกลุ่มอำเภอที่เลือก: 37396 แปลง
จำนวนแปลงเกษตรตั้งต้น: 37396 แปลง


In [68]:
from shapely.validation import make_valid # นำเข้าฟังก์ชัน fix geometry

# 2.---Fix Geometry (เหมือน Fix Geometry ใน QGIS) ---
print("กำลังตรวจสอบและซ่อมแซม Geometry (Fix Geometry)...")

# ตรวจสอบว่ามีแปลงที่ Invalid หรือไม่
invalid_count = (~farms.is_valid).sum()
print(f"พบ Geometry ที่มีปัญหา: {invalid_count:,} แปลง")

if invalid_count > 0:
    # ใช้ .make_valid() เพื่อซ่อมแซมรูปทรง)
    farms['geometry'] = farms['geometry'].apply(lambda geom: make_valid(geom) if not geom.is_valid else geom)
    print("ซ่อมแซม Geometry เรียบร้อยแล้ว")
else:
    print("Geometry ทั้งหมดสมบูรณ์")
# --------------------------------------------------------

กำลังตรวจสอบและซ่อมแซม Geometry (Fix Geometry)...
พบ Geometry ที่มีปัญหา: 0 แปลง
Geometry ทั้งหมดสมบูรณ์


In [ ]:
import math
import time
import requests
import geopandas as gpd

# 3. ฟังก์ชันดึง API แบบละเอียด (ป้องกันข้อมูลหลุด)
def get_bbox(gdf):
    minx, miny, maxx, maxy = gdf.total_bounds
    return f"{minx},{miny},{maxx},{maxy}"

bbox = get_bbox(farms)
api_key = "YOUR_API-KEY"
collection_id = "66b07c8034c2963abcdb9bb8"

# 1. เช็กจำนวนทั้งหมดอีกครั้ง
initial_url = f"https://api-gateway.gistda.or.th/api/2.0/resources/features/flood-freq?api_key={api_key}&bbox={bbox}&collectionCreatedBy={collection_id}&limit=1&offset=0"
res = requests.get(initial_url).json()
total_features = res.get('numberMatched', 0)

# 2. ปรับ Limit ให้เล็กลง (ใช้ 1000 หรือ 2000 เพื่อความเสถียรสูงสุด)
limit = 1000 
total_pages = math.ceil(total_features / limit)
all_features = []

print(f"--- เริ่มดึงข้อมูลแบบละเอียดสูงสุด (Target: {total_features:,} รายการ) ---")

for i in range(total_pages):
    offset = i * limit
    url = f"https://api-gateway.gistda.or.th/api/2.0/resources/features/flood-freq?api_key={api_key}&bbox={bbox}&collectionCreatedBy={collection_id}&limit={limit}&offset={offset}"
    
    # ระบบ Retry: ถ้าเจอค่าว่างให้ลองใหม่สูงสุด 3 ครั้ง
    for attempt in range(1, 4):
        try:
            response = requests.get(url, timeout=60)
            data = response.json()
            features = data.get('features', [])
            
            if len(features) > 0:
                all_features.extend(features)
                print(f"รอบที่ {i+1}/{total_pages}: ดึงได้ {len(features):,} รายการ | สะสมรวม: {len(all_features):,}")
                break # ได้ข้อมูลแล้ว ไปรอบถัดไป
            else:
                # ถ้าเป็นหน้าท้ายๆ แล้วไม่มีข้อมูลจริงๆ ให้หยุด
                if offset >= total_features:
                    break
                print(f"รอบที่ {i+1} (พยายามครั้งที่ {attempt}): เจอค่าว่าง... กำลังลองใหม่")
                time.sleep(2)
        except Exception as e:
            print(f"รอบที่ {i+1} พลาด: {e}")
            time.sleep(3)
            
    time.sleep(0.5) #กันโดน Block

# 3. ตรวจสอบและสร้าง GeoDataFrame
if len(all_features) > 0:
    flood_gdf = gpd.GeoDataFrame.from_features(all_features, crs="EPSG:4326")
    flood_gdf.to_file(r"E:\PythonFor_Flood\flood_data_temp.gpkg", driver="GPKG")
    print(f"ดึงเสร็จสิ้น! ได้ข้อมูลทั้งหมด: {len(flood_gdf):,} จาก {total_features:,} รายการ")
    print(flood_gdf['freq'].value_counts().sort_index())
else:
    print("ไม่สามารถดึงข้อมูลได้เลย")

--- เริ่มดึงข้อมูลแบบละเอียดสูงสุด (Target: 9,128 รายการ) ---
รอบที่ 1/10: ดึงได้ 1,000 รายการ | สะสมรวม: 1,000
รอบที่ 2/10: ดึงได้ 1,000 รายการ | สะสมรวม: 2,000
รอบที่ 3/10: ดึงได้ 1,000 รายการ | สะสมรวม: 3,000
รอบที่ 4/10: ดึงได้ 1,000 รายการ | สะสมรวม: 4,000
รอบที่ 5/10: ดึงได้ 1,000 รายการ | สะสมรวม: 5,000
รอบที่ 6/10: ดึงได้ 1,000 รายการ | สะสมรวม: 6,000
รอบที่ 7/10: ดึงได้ 1,000 รายการ | สะสมรวม: 7,000
รอบที่ 8/10: ดึงได้ 1,000 รายการ | สะสมรวม: 8,000
รอบที่ 9 (พยายามครั้งที่ 1): เจอค่าว่าง... กำลังลองใหม่
รอบที่ 9/10: ดึงได้ 1,000 รายการ | สะสมรวม: 9,000
รอบที่ 10/10: ดึงได้ 128 รายการ | สะสมรวม: 9,128

✅ ดึงเสร็จสิ้น! ได้ข้อมูลทั้งหมด: 9,128 จาก 9,128 รายการ
freq
1     835
2    3645
3    4300
4     348
Name: count, dtype: int64


In [73]:
# 4. แปลงพิกัดเป็น UTM เพื่อความแม่นยำ
farms_utm = farms.to_crs(epsg=32647)
flood_utm = flood_gdf.to_crs(epsg=32647)
#เช็กว่า CRS ตรงกันจริงไหมก่อน Join
print(f"Farms CRS: {farms_utm.crs}")
print(f"Flood CRS: {flood_utm.crs}")



Farms CRS: EPSG:32647
Flood CRS: EPSG:32647


In [74]:

# 5.sjoin แบบ left: เก็บทุกแปลงไว้ (Eliminate missing data จากการ Intersect)
joined_data = gpd.sjoin(farms_utm, flood_utm[['freq', 'geometry']], how='left', predicate='intersects')

#ลองเช็กจำนวนแปลงที่ "ท่วม" เทียบกับ "ทั้งหมด"
total = len(joined_data)
flooded = joined_data['freq'].notna().sum()



print("--- ผลลัพธ์หลังการ Join (ยังไม่ Clean) ---")
print(joined_data[['act','type_name', 'detail_nam','plant_date','year_act','p_name', 'a_name','t_name','moo','freq']].head(5))
print(f"จากทั้งหมด {total:,} แปลง | พบพื้นที่น้ำท่วม {flooded:,} แปลง")

--- ผลลัพธ์หลังการ Join (ยังไม่ Clean) ---
           act type_name detail_nam plant_date year_act           p_name  \
145  163730806      ข้าว   ข้าวเจ้า   20220504     2565  พระนครศรีอยุธยา   
145  163730806      ข้าว   ข้าวเจ้า   20220504     2565  พระนครศรีอยุธยา   
185  166191184      ข้าว   ข้าวเจ้า   20220531     2565  พระนครศรีอยุธยา   
185  166191184      ข้าว   ข้าวเจ้า   20220531     2565  พระนครศรีอยุธยา   
185  166191184      ข้าว   ข้าวเจ้า   20220531     2565  พระนครศรีอยุธยา   

     a_name t_name moo  freq  
145  ผักไห่   อมฤต   1     3  
145  ผักไห่   อมฤต   1     4  
185  ผักไห่   อมฤต   9     4  
185  ผักไห่   อมฤต   9     3  
185  ผักไห่   อมฤต   9     3  
จากทั้งหมด 73,923 แปลง | พบพื้นที่น้ำท่วม 73,923 แปลง


In [75]:
# 6. เรียงลำดับความสำคัญ: 
# - เลขประจำกิจกรรม (act) 
# - ปีล่าสุด (year_act) -> เรียงจากมากไปน้อย (False)
# - ความเสี่ยงสูงสุด (freq) -> เรียงจากมากไปน้อย (False)
joined_data_sorted = joined_data.sort_values(
    by=['act', 'year_act', 'freq'], 
    ascending=[True, False, False]
)

# 7. ลบตัวซ้ำ (Deduplicate)
# จะเก็บเฉพาะ "แถวแรก" ของแต่ละ 'act' (ซึ่งก็คือแถวที่มีปีใหม่สุดและ freq สูงสุด)
cleaned_farms = joined_data_sorted.drop_duplicates(subset=['act'], keep='first').copy()

print(f"--- สรุปการจัดการข้อมูลซ้ำ ---")
print(f"จำนวนข้อมูลที่ Join มาทั้งหมด: {len(joined_data):,} แถว")
print(f"จำนวนแปลงเกษตรที่ Clean แล้ว: {len(cleaned_farms):,} แปลง")
print(f"จำนวนแปลงที่ลดลงไป (ตัวซ้ำ): {len(joined_data) - len(cleaned_farms):,} แถว")

# ตรวจสอบว่าจำนวนแปลงกลับมาใกล้เคียงค่าตั้งต้นหรือยัง (ควรกลับมาเท่ากับจํานวนขั้นตอนแรก)

--- สรุปการจัดการข้อมูลซ้ำ ---
จำนวนข้อมูลที่ Join มาทั้งหมด: 73,923 แถว
จำนวนแปลงเกษตรที่ Clean แล้ว: 37,396 แปลง
จำนวนแปลงที่ลดลงไป (ตัวซ้ำ): 36,527 แถว


In [77]:
# 8. จัดกลุ่มพืช 5 กลุ่ม (Group Mapping)
def crop_classification(crop):
    if pd.isna(crop): return '5-อื่นๆ'
    crop = str(crop)
    if 'ข้าว' in crop: return '1-ข้าว'
    if 'พืชไร่' in crop: return '2-พืชไร่'
    if 'พืชสวน' in crop: return '3-พืชสวน'
    if 'พืชผัก' in crop: return '4-พืชผัก'
    return '5-อื่นๆ' # เพิ่มบรรทัดนี้กันเหนียวเผื่อข้อมูลไม่เข้าเงื่อนไขด้านบน
    

cleaned_farms['crop_group'] = cleaned_farms['type_name'].apply(crop_classification)

#  วิเคราะห์สถานะความเสี่ยง (ใช้ปีข้อมูล และความถี่น้ำท่วม)
# นิยาม: ถ้า freq มีค่า (ไม่เป็น NaN) = มีความเสี่ยง
def assess_risk(row):
    if pd.isna(row['freq']):
        return '0' #-ไม่เสี่ยง
    if row['freq'] >= 4: return '4' #-เสี่ยงสูงมาก
    if row['freq'] >= 3: return '3'#-เสี่ยงสูง
    if row['freq'] >= 2: return '2'#-เสี่ยงปานกลาง
    if row['freq'] >= 1: return '1'#-เสี่ยงน้อย

cleaned_farms['risk_level_new'] = cleaned_farms.apply(assess_risk, axis=1)

print("--- ตัวอย่างผลลัพธ์การจัดกลุ่มพืชและความเสี่ยง ---")
print(cleaned_farms[['act','type_name','detail_nam', 'crop_group', 'risk_level_new']].head(10))

--- ตัวอย่างผลลัพธ์การจัดกลุ่มพืชและความเสี่ยง ---
              act type_name detail_nam crop_group risk_level_new
133545  123115255      ข้าว   ข้าวเจ้า     1-ข้าว              4
119303  123188568      ข้าว   ข้าวเจ้า     1-ข้าว              4
140692  123190978      ข้าว   ข้าวเจ้า     1-ข้าว              4
90848   123235786      ข้าว   ข้าวเจ้า     1-ข้าว              4
24930   123253109      ข้าว   ข้าวเจ้า     1-ข้าว              4
57058   123300247      ข้าว   ข้าวเจ้า     1-ข้าว              3
66750   123300384      ข้าว   ข้าวเจ้า     1-ข้าว              4
111159  123300559      ข้าว   ข้าวเจ้า     1-ข้าว              4
75124   123301178      ข้าว   ข้าวเจ้า     1-ข้าว              4
17753   123301229      ข้าว   ข้าวเจ้า     1-ข้าว              4


In [78]:
# 9. สรุปข้อมูลเป็นตาราง (แทนการใช้กราฟเพื่อเลี่ยงปัญหาฟอนต์ภาษาไทย)
print("--- ตารางสรุปจำนวนแปลงแยกตามระดับความเสี่ยง ---")
summary_table = cleaned_farms['risk_level_new'].value_counts().sort_index()
print(summary_table)



--- ตารางสรุปจำนวนแปลงแยกตามระดับความเสี่ยง ---
risk_level_new
1      669
2      840
3     3197
4    32690
Name: count, dtype: int64


In [79]:
# 10. คัดเลือกเฉพาะ Field ที่ต้องการ (จัดระเบียบข้อมูล)
export_cols = [
    'p_name', 'a_name', 't_name', 'moo', 'act', 'year_act',
    'detail_nam', 'crop_group', 'plant_date', 'produce_da', 
    'freq', 'risk_level_new', 'geometry'
]
final_output = cleaned_farms[[c for c in export_cols if c in cleaned_farms.columns]].copy()

#  ส่งออกไฟล์
print("\n--- กำลังส่งออกไฟล์ ---")

# ส่งออกเป็น GeoPackage
final_output.to_file(r"E:\PythonFor_Flood\Flood_Analysis_Ayutthaya.gpkg", driver="GPKG")

# ส่งออก.csv 
final_output.drop(columns='geometry').to_csv(r"E:\PythonFor_Flood\Flood_Risk_Report_Ayutthaya.csv", index=False, encoding='utf-8-sig')

print("ส่งออกเป็นไฟล์ .csv เรียบร้อยแล้ว (สามารถใช้ Excel เปิดได้เลย)")


--- กำลังส่งออกไฟล์ ---
ส่งออกเป็นไฟล์ .csv เรียบร้อยแล้ว (สามารถใช้ Excel เปิดได้เลย)
